# Classification of UN Speaches

In [1]:
import pandas as pd
import re

import nltk
nltk.download('stopwords')
nltk.download("wordnet")
nltk.download('omw-1.4')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix

from pycaret.classification import *

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vscode/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/vscode/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Step 1 - Load the Dataset

In [2]:
df = pd.read_csv("data/source/un-general-debates-blueprint.csv.gz")
print(df.shape)
df.sample(5)

(7507, 7)


,session,year,country,country_name,speaker,position,text
998,33,1978,DOM,Dominican Republic,Jimenez,NaN,﻿\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
6469,65,2010,MWI,Malawi,Bingu Wa Mutharika,President,I am delighted to address \nthe sixty-fifth se...
5397,60,2005,ALB,Albania,Besnik Mustafaj,Minister for Foregn Affairs,"Let\nme first convey, on behalf of the people ..."
2023,40,1985,DOM,Dominican Republic,Mr. Vega Imbert,NaN,This session of the General Assembly is of a v...
5144,58,2003,OMN,Oman,Yousef Bin Al-Alawi Bin Abdulla,Minister for Foreign Affairs,"﻿Allow\nme, Sir, on behalf of the Sultanate of..."


In [3]:
### Only pick the five countries in the UN Security Cpuncil

SECURITY_COUNCIL = ["USA", "FRA", "GBR", "RUS", "CHN"]

df_council = df[df["country"].isin(SECURITY_COUNCIL)][["country", "text"]].reset_index(drop=True)
print(df_council.shape)
df_council.head()

(228, 2)


,country,text
0,FRA,"84.\t Within one month, when we celebrate the..."
1,GBR,"110.\t Mr. President, I should like first to s..."
2,USA,1.\t It is my privilege to extend to you once ...
3,CHN,"1.\t Mr. President, it is my pleasant duty, o..."
4,FRA,34.\tIt is toward Asia where all the problems ...


In [4]:
### Check for missing values

df_council.isna().sum()

country    0
text       0
dtype: int64

## Step 2 - Normalize the Text

In [5]:
def normalize(text, stemm="porter", lemm="wordnet", stopwords="nltk"):

    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    list_words = text.split()

    ## remove Stopwords
    if stopwords == "nltk":
        list_stopwords = nltk.corpus.stopwords.words("english")
        list_words = [word for word in list_words if word not in list_stopwords]
    elif stopwords == "spacy":
        pass
    elif stopwords == "gensim":
        pass
    else:
        pass
                
    ## Stemming (remove -ing, -ly, ...)
    if stemm == "porter":
        ps = nltk.stem.porter.PorterStemmer()
        list_words = [ps.stem(word) for word in list_words]
    elif stemm == "snowball":
        ps = nltk.stem.snowball.SnowballStemmer("english")
        list_words = [ps.stem(word) for word in list_words]
    else:
        pass
                
    ## Lemmatisation (convert the word into root word)
    if lemm == "wordnet":
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        list_words = [lem.lemmatize(word) for word in list_words]
    else:
        pass

    return " ".join(list_words)

In [6]:
df_council["normalized"] = df_council["text"].apply(normalize, args=["snowball", "wordnet", "nltk"])

df_council.head(5)

,country,text,normalized
0,FRA,"84.\t Within one month, when we celebrate the...",84 within one month celebr twentyfifth anniver...
1,GBR,"110.\t Mr. President, I should like first to s...",110 mr presid like first say glad sit presid a...
2,USA,1.\t It is my privilege to extend to you once ...,1 privileg extend warm congratul unit state de...
3,CHN,"1.\t Mr. President, it is my pleasant duty, o...",1 mr presid pleasant duti behalf govern deleg ...
4,FRA,34.\tIt is toward Asia where all the problems ...,34 toward asia problem world ferment whose anc...


## Step 3 - Split the Data into Training and Test 

In [7]:
X = df_council["normalized"]
y = df_council["country"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(182,) (182,)
(46,) (46,)


## Step 4 - Vectorize the Training and Test Data

In [8]:
tfidf = TfidfVectorizer()

X_train_vectorized = tfidf.fit_transform(X_train)    # Fit with the train data
X_test_vectorized = tfidf.transform(X_test)          # used the same vectorize to transform the test data

X_names = tfidf.get_feature_names()

df_X_train_vectorized =  pd.DataFrame(X_train_vectorized.toarray(), columns=X_names, index=X_train.index)

print(df_X_train_vectorized.shape)

df_X_test_vectorized =  pd.DataFrame(X_test_vectorized.toarray(), columns=X_names, index=X_test.index)

print(df_X_test_vectorized.shape)

(182, 10681)
(46, 10681)


## Step 5 - Dimensionality Reduction using Chi-squared

In [2]:
ch2 = SelectKBest(chi2, k=200)
X_train_chi2 = ch2.fit_transform(X_train_vectorized, y_train)
X_test_chi2 = ch2.transform(X_test_vectorized)

# feature_names = X_names[ch2.get_support()]

df_X_train_chi2 =  pd.DataFrame(X_train_chi2.toarray(),  index=X_train.index)

print(df_X_train_chi2.shape)

df_X_test_chi2 =  pd.DataFrame(X_test_chi2.toarray(),  index=X_test.index)

print(df_X_test_chi2.shape)

NameError: name 'SelectKBest' is not defined

In [10]:
df_train_chi2 = pd.concat([y_train, df_X_train_chi2], axis=1)

df_test_chi2 = pd.concat([y_test, df_X_test_chi2], axis=1)



In [11]:
df_train_chi2.head()

,country,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
52,USA,0.0,0.0,0.014876,0.008564,0.029409,0.016679,0.058248,0.009984,0.033135,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.017030,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.014494,0.0,0.013062,0.000000,0.000000,0.000000,0.000000,0.042967,0.038510,0.0,0.008865,0.013684,0.010559,0.000000,0.026529,0.000000,0.012772,0.019968,0.015008,0.0,0.000000,0.0,0.041122,0.024971,0.0,0.0,0.025256,0.009744,0.0,0.000000,0.0,0.000000,0.000000,0.009074,0.0,0.0,0.0,0.041122,0.000000,0.000000,0.0,0.0,0.0,0.007280,0.000000,0.000000,0.0,0.0,0.0,0.0,0.042093,0.013684,0.019726,0.0,0.012679,0.161338,0.000000,0.184261,0.000000,0.0,0.097284,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.012495,0.0,0.068512,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.008183,0.012059,0.000000,0.0,0.0,0.058082,0.019606,0.000000,0.314500,0.00000,0.028332,0.000000,0.072826,0.0,0.024811,0.014166,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.224643,0.0,0.000000,0.016184,0.0,0.000000,0.053268,0.027152,0.0,0.000000,0.0,0.034061,0.000000,0.008277,0.000000,0.0,0.0,0.037215,0.000000,0.0,0.000000,0.0,0.0,0.0,0.012586,0.000000,0.000000,0.0,0.024139,0.000000,0.000000,0.0,0.000000,0.082893,0.000000,0.000000,0.000000,0.000000,0.144325,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.013684,0.0,0.0,0.000000,0.0,0.0,0.050512,0.0,0.237479,0.014134,0.000000,0.000000,0.000000,0.016854,0.0,0.000000,0.0,0.087035,0.026124,0.0,0.040897,0.0,0.056103
10,GBR,0.0,0.0,0.000000,0.020450,0.011704,0.000000,0.013909,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.020333,0.0,0.0,0.000000,0.097549,0.042538,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.042302,0.015138,0.021026,0.000000,0.025650,0.091958,0.0,0.000000,0.000000,0.000000,0.037523,0.063349,0.000000,0.000000,0.000000,0.017919,0.0,0.022918,0.0,0.032732,0.019876,0.0,0.0,0.180924,0.093070,0.0,0.000000,0.0,0.015363,0.010284,0.000000,0.0,0.0,0.0,0.057280,0.000000,0.020123,0.0,0.0,0.0,0.017385,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.016337,0.000000,0.0,0.000000,0.069346,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.051916,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.014812,0.029312,0.043192,0.018762,0.0,0.0,0.100167,0.023409,0.000000,0.122611,0.00000,0.016913,0.000000,0.000000,0.0,0.000000,0.067653,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.045979,0.0,0.000000,0.038645,0.0,0.018942,0.072684,0.000000,0.0,0.000000,0.0,0.040666,0.000000,0.000000,0.000000,0.0,0.0,0.008886,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.067740,0.0,0.019214,0.000000,0.017159,0.0,0.000000,0.061856,0.000000,0.000000,0.000000,0.000000,0.054828,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.032675,0.0,0.0,0.000000,0.0,0.0,0.040205,0.0,0.099621,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.015595,0.0,0.024415,0.0,0.000000
95,GBR,0.0,0.0,0.000000,0.007661,0.017538,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.020499,0.021129,0.0,0.0,0.015234,0.0,0.0,0.020499,0.190022,0.031870,0.0,0.0,0.0,0.011341,0.080615,0.012965,0.0,0.046737,0.021129,0.000000,0.047259,0.000000,0.019217,0.155017,0.0,0.000000,0.000000,0.009445,0.000000,0.059327,0.009752,0.011425,0.000000,0.000000,0.0,0.000000,0.0,0.042915,0.007446,0.0,0.0,0.082837,0.01

In [12]:
df_test_chi2.head()

,country,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
160,GBR,0.0,0.0,0.024494,0.014101,0.032283,0.0,0.019182,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.029333,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.043015,0.000000,0.0,0.000000,0.0,0.011792,0.063410,0.00000,0.000000,0.0,0.000000,0.0,0.032762,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.011285,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.014182,0.014940,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.035964,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.016240,0.000000,0.000000,0.159395,0.0,0.000000,0.342333,0.0,0.0,0.0,0.0,0.000000,0.071598,0.000000,0.000000,0.000000,0.000000,0.0,0.042304,0.000000,0.0,0.015117,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.085010,0.000000,0.0,0.243073,0.0,0.058313,0.0,0.026648,0.0,0.000000,0.046651,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.063410,0.0,0.0,0.000000,0.0,0.000000,0.012530,0.0,0.000000,0.000000,0.000000,0.00000,0.023665,0.000000,0.000000,0.000000,0.0,0.012255,0.0,0.0,0.0,0.000000,0.000000,0.087296,0.020725,0.0,0.015570,0.000000,0.013249,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.129624,0.0,0.000000,0.00000,0.000000,0.071598,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.264210,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.130282,0.00000,0.000000,0.033671,0.0,0.0
178,CHN,0.0,0.0,0.000000,0.000000,0.012744,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.022140,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.016483,0.351482,0.056529,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.121028,0.158538,0.00000,0.000000,0.0,0.013727,0.0,0.327647,0.056694,0.0,0.000000,0.000000,0.0,0.0,0.0,0.035640,0.064925,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.011197,0.000000,0.0,0.0,0.0,0.017820,0.0,0.109557,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.035576,0.0,0.0,0.0,0.010945,0.000000,0.000000,0.000000,0.000000,0.125848,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.017511,0.070044,0.000000,0.0,0.000000,0.000000,0.0,0.035805,0.0,0.0,0.017932,0.0,0.000000,0.000000,0.020429,0.0,0.0,0.000000,0.089211,0.0,0.141850,0.0,0.018416,0.0,0.126236,0.0,0.032255,0.073665,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.208603,0.0,0.0,0.010520,0.0,0.000000,0.000000,0.0,0.000000,0.134369,0.000000,0.01107,0.000000,0.032279,0.050185,0.000000,0.0,0.067733,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.065451,0.0,0.049173,0.019170,0.000000,0.0,0.000000,0.0,0.013555,0.000000,0.0,0.000000,0.000000,0.0,0.051171,0.0,0.018225,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.028172,0.017789,0.0,0.000000,0.020237,0.0,0.0,0.000000,0.0,0.133506,0.036750,0.0,0.015145,0.0,0.0,0.000000,0.0,0.0,0.020572,0.00000,0.065199,0.008861,0.0,0.0
130,GBR,0.0,0.0,0.016752,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0133,0.024404,0.0,0.019179,0.0,0.0,0.0,0.110413,0.020062,0.0,0.0,0.0,0.000000,0.025373,0.016323,0.000000,0.000000,0.013300,0.0,0.000000,0.0,0.008065,0.173474,0.00000,0.000000,0.0,0.000000,0.0,0.351045,0.000000,0.0,0.000000,0.016901,0.0,0.0,0.0,0.038592,0.000000,0.0,0.0,0.000000,0.032920,0.0,0.0,0.0,0.0,0.096997,0.040873,0.0,0.0,0.0,0.038592,0.0,0.000000,0.0,0.0,0.0,0.065591,0.275505,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000

## Step 6 - Classification using sklearn LinearSVC

In [13]:
lsvc = LinearSVC(verbose=0)
print(lsvc)

LinearSVC()


In [14]:
lsvc.fit(X_train_chi2, y_train)
score = lsvc.score(X_train_chi2, y_train)
print("Score: ", score)

Score:  0.9945054945054945


In [15]:
ypred = lsvc.predict(X_test_chi2)

cm = confusion_matrix(y_test, ypred)
print(cm)

[[ 7  0  0  0  0]
 [ 0  7  0  0  0]
 [ 0  0  9  0  1]
 [ 0  0  0 12  0]
 [ 0  0  0  0 10]]


## Step 7 - Classification using PyCaret

In [16]:
experiment = setup(data=df_train_chi2, target="country", test_data=df_test_chi2, 
    silent=True, fold=5, session_id=123)

,Description,Value
0,session_id,123
1,Target,country
2,Target Type,Multiclass
3,Label Encoded,"CHN: 0, FRA: 1, GBR: 2, RUS: 3, USA: 4"
4,Original Data,"(182, 201)"
5,Missing Values,False
6,Numeric Features,199
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


In [21]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9836,0.9991,0.9836,0.9849,0.9836,0.9795,0.9799,0.1380
ridge,Ridge Classifier,0.9673,0.0000,0.9686,0.9716,0.9669,0.9591,0.9605,0.0080
rf,Random Forest Classifier,0.9670,0.9989,0.9664,0.9722,0.9667,0.9587,0.9601,0.1660
lr,Logistic Regression,0.9616,0.9927,0.9621,0.9652,0.9609,0.9519,0.9531,0.0180
lightgbm,Light Gradient Boosting Machine,0.9616,0.9990,0.9621,0.9649,0.9611,0.9519,0.9529,0.1100
gbc,Gradient Boosting Classifier,0.9279,0.9880,0.9279,0.9378,0.9300,0.9099,0.9116,0.7000
knn,K Neighbors Classifier,0.9228,0.9893,0.9229,0.9302,0.9226,0.9034,0.9055,0.0160
svm,SVM - Linear Kernel,0.9227,0.0000,0.9207,0.9409,0.9191,0.9030,0.9087,0.0140
dt,Decision Tree Classifier,0.8619,0.9141,0.8598,0.8796,0.8579,0.8271,0.8322,0.0100
lda,Linear Discriminant Analysis,0.8462,0.9444,0.8457,0.8644,0.8450,0.8076,0.8120,0.0140


In [22]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [23]:
predict_model(best)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


,1,2,3,4,5,6,7,8,9,10,...,194,195,196,197,198,199,0_0.0,country,Label,Score
0,0.000000,0.024494,0.014101,0.032283,0.000000,0.019182,0.000000,0.000000,0.000000,0.000000,...,0.130282,0.000000,0.000000,0.033671,0.000000,0.000000,1.0,GBR,GBR,0.49
1,0.000000,0.000000,0.000000,0.012744,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020572,0.000000,0.065199,0.008861,0.000000,0.000000,1.0,CHN,CHN,0.83
2,0.000000,0.016752,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.008910,0.029420,0.000000,0.015352,0.000000,0.000000,1.0,GBR,GBR,0.72
3,0.000000,0.000000,0.064146,0.024476,0.000000,0.014543,0.000000,0.000000,0.000000,0.000000,...,0.059264,0.000000,0.000000,0.017018,0.000000,0.000000,1.0,RUS,RUS,0.75
4,0.000000,0.011423,0.013152,0.015055,0.000000,0.000000,0.030665,0.008481,0.000000,0.000000,...,0.012151,0.040120,0.000000,0.015702,0.000000,0.000000,1.0,GBR,GBR,0.47
5,0.033253,0.000000,0.021486,0.012297,0.000000,0.000000,0.000000,0.013855,0.000000,0.000000,...,0.059552,0.000000,0.000000,0.017101,0.000000,0.000000,1.0,RUS,RUS,0.76
6,0.000000,0.019977,0.023002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.017541,0.000000,0.045769,0.000000,0.000000,1.0,GBR,GBR,0.65
7,0.000000,0.000000,0.043109,0.008224,0.000000,0.009774,0.016752,0.018533,0.000000,0.000000,...,0.026552,0.021917,0.000000,0.080060,0.000000,0.000000,1.0,FRA,FRA,0.68
8,0.000000,0.023913,0.000000,0.000000,0.000000,0.009363,0.168518,0.044388,0.000000,0.000000,...,0.063595,0.010499,0.000000,0.021914,0.000000,0.000000,1.0,USA,USA,0.72
9,0.000000,0.000000,0.000000,0.019128,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.093100,0.000000,0.000000,1.0,GBR,GBR,0.48
